# Step 0 - import relation module

In [1]:
%matplotlib inline
import os,sys
import datacube
from datacube.config import LocalConfig
import datetime
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
sys.path.append(os.environ['odc_config'])
dc = datacube.Datacube(app = 'my_app', config = '/home/localuser/.datacube.conf')
#import 會用到的模組

# Step 1 - loading image


In [ ]:
platform = "LANDSAT_8"
product = "ls8_C1_sr_taiwan"

base_lat = 22.98322
base_lon = 120.22082

#base_lat、base_lon代表要取的那點的座標

lon = (base_lon + 0.001, base_lon - 0.001)
lat = (base_lat + 0.001, base_lat - 0.001)

#將經緯度範圍設定為+-0.001的範圍

date_range =(datetime.datetime(2010,1,1), datetime.datetime(2019,12,31))

platform = 'LANDSAT_8'
product = 'ls8_C1_sr_taiwan'  

desired_bands = ['red','green','blue','nir','swir1','swir2','pixel_qa']  

#取出資料
landsat = dc.load(product = product,platform = platform,lat = lat,lon = lon, time = date_range,measurements = desired_bands,group_by = 'solar_day')

# Step 2 - extract data according to coordinates and filter by pixel quality

In [ ]:
pixel_coordinates = {"latitude":base_lat,"longitude": base_lon}
pixel = landsat.sel(**pixel_coordinates, method = 'nearest')
#根據坐標取出資料

In [ ]:
point = pixel.where(pixel.pixel_qa == 322)
#篩選pixel_qa為322的資料出來

# Step 3 - get ndvi values

In [ ]:
ndvi = (point.nir - point.red)/(point.nir + point.red)
#計算ndvi

# Step 4 - output to pandas dataframe

In [ ]:
result = pd.DataFrame({'time':ndvi.time.values})
result['ndvi'] = ndvi.values
#做成表格

In [ ]:
result = result[~result['ndvi'].isin(['NaN'])]

# Step 5 - plot ndvi values by time

In [ ]:
#計算繪圖用資料
result2 = pd.DataFrame({'time':ndvi.time.values})
result2['ndvi'] = ndvi.values
result2 = result2[~result2['ndvi'].isin(['NaN'])]

In [ ]:
#用seaborn繪製散布圖
sns.set()

plt.subplots(figsize=(17,5))
result.set_index("time")['ndvi'].plot(style='k--')
sns.scatterplot(x="time", y="ndvi", data=result2)
#根據時間繪圖

In [ ]:
#指數加權函數分析
import time
sns.set()
plt.subplots(figsize=(17,5))
plt.xlim(pd._libs.tslib.Timestamp('2013-01-01 00:00:00'), pd._libs.tslib.Timestamp('2020-01-01 00:00:00')) 

test = result2.set_index("time")['ndvi'].rolling(window=60,min_periods=50).mean()
#套用指數分析
ewma60 = result2.set_index("time")['ndvi'].ewm(span=3).mean()
ewma60.plot(style='k--')

sns.scatterplot(x="time", y="ndvi", data=result2)


# Step 6 - output to csv

In [ ]:
result.to_csv('ndvi_value.csv',index=True,sep=',')
#輸出成csv檔